In [54]:
import numpy as np
from scipy.stats import norm, chi2, poisson
import pandas as pd

Exercise 6

In [ ]:
n = 1919
n1 = 922
n2 = 997
theta_mle = n1/n
se_mle = np.sqrt((theta_mle * (1 - theta_mle))/n)
theta_0 = 0.5
alpha = 0.05
z = norm.ppf(1-alpha/2)
confidence_interval = (theta_mle - z * se_mle, theta_mle + z * se_mle)
W = (theta_mle - theta_0) / se_mle
p_value = 2 * (1-norm.cdf(np.abs(W)))

print('Estimated theta: \t %.3f' % theta_mle)
print('Estimated SE: \t\t %.3f' % se_mle)
print('95%% confidence interval: (%.3f, %.3f)' % confidence_interval)
print('Wald statistic: \t %.3f' % W)
print('p-value: \t\t %.3f' % p_value)

Estimated theta: 	 0.480
Estimated SE: 		 0.011
95% confidence interval: (0.458, 0.503)
Wald statistic: 	 -1.713
p-value: 		 0.087


Exercise 7

In [ ]:
twain_ratios = np.array([.225, .262, .217, .240, .230, .229, .235, .217])
snod_ratios = np.array([.209, .205, .196, .210, .202, .207, .224, .223, .220, .201])
mean_1 = np.mean(twain_ratios)
mean_2 = np.mean(snod_ratios)
diff_hat = mean_1 - mean_2
var_1 = np.var(twain_ratios)/len(twain_ratios)
var_2 = np.var(snod_ratios)/len(snod_ratios)
se_hat = np.sqrt(var_1 + var_2)
alpha = 0.05
z = norm.ppf(1-alpha/2)
ci = (diff_hat - z * se_hat, diff_hat + z * se_hat)
W = diff_hat/se_hat
p_value = 2 * (norm.cdf(-np.abs(W)))

print('Estimated difference of means:\t %.3f' % diff_hat)
print('Estimated SE: \t\t\t %.3f' % se_hat)
print('95%% confidence interval:\t (%.3f, %.3f)' % ci)
print('Wald statistic: \t\t %.3f' % W)
print('Wald test p-value: \t\t %.5f' % p_value)


Estimated difference of means:	 0.022
Estimated SE: 			 0.006
95% confidence interval:	 (0.011, 0.033)
Wald statistic: 		 3.945
Wald test p-value: 		 0.00008


In [ ]:
#Permutation test
B = 10000
ratios = np.concatenate([twain_ratios, snod_ratios])
n = len(twain_ratios)
count = 0
for _ in range(B):
  np.random.shuffle(ratios)
  x, y = ratios[:n], ratios[n:]
  diff = np.mean(x) - np.mean(y)
  if diff > diff_hat:
    count += 1
p_value = count/B

print('Permutation test p-value: \t\t %.5f' % p_value)


Permutation test p-value: 		 0.00040


Exercise 10

In [ ]:
#Multiple testing // Bonferroni method
#Testing weekly death rate between the 2 populations
df = pd.DataFrame({
    'Chinese': [55, 33, 70, 49],
    'Jewish': [141, 145, 139, 161]
}, index = [-2, -1, 1, 2])
total = df.sum()
fracs = df / total
deltas = fracs['Chinese'] - fracs['Jewish']
#fracs.sem(axis = 1, ddof = 0)
std_errs = np.sqrt(np.sum(fracs * (1. - fracs)/total, axis = 1))
W = (deltas - 0)/std_errs
p_values = 2 * norm.cdf(-np.abs(W))
bonferroni_p_values = np.minimum(len(p_values) * p_values, 1.)
result = pd.DataFrame({
    'p_values': p_values,
    'Bonferroni p_values': bonferroni_p_values
}, index = [-2, -1, 1, 2])
result




,p_values,Bonferroni p_values
-2,0.478749,1.000000
-1,0.004608,0.018430
1,0.006768,0.027071
2,0.274853,1.000000


In [ ]:
#χ2 test
#Testing populations separately, 2 parameters for population: before and after ratios // multinomial
#Null is before ratio = after ratio = 1/2
#Chinese
total = df.sum()
p = 1/4 #p1=p2=p3=p4
expected_ch = p * total['Chinese']
numerator =[(df['Chinese'].loc[i] - expected_ch)**2 for i in [-2, -1, 1, 2]]

pearson_chinese = (numerator/expected_ch).sum()
chi2_95 = chi2.ppf(0.95, 3)
p_value = 1 - chi2.cdf(pearson_chinese, 3)
print('Test statistic:\t\t\t\t%.3f ' % pearson_chinese)
print('95%% percentile chi squared with 3 df:\t%.3f' % chi2_95)
print('p-value for different distributions:\t%.3f' % p_value)

Test statistic:				13.580 
95% percentile chi squared with 3 df:	7.815
p-value for different distributions:	0.004


In [ ]:
#Jewish
total = df.sum()
p = 1/4 #p1=p2=p3=p4
expected_ch = p * total['Jewish']
numerator =[(df['Jewish'].loc[i] - expected_ch)**2 for i in [-2, -1, 1, 2]]

pearson_jewish = (numerator/expected_ch).sum()
chi2_95 = chi2.ppf(0.95, 3)
p_value = 1 - chi2.cdf(pearson_jewish, 3)
print('Test statistic:\t\t\t\t%.3f ' % pearson_jewish)
print('95%% percentile chi squared with 3 df:\t%.3f' % chi2_95)
print('p-value for different distributions:\t%.3f' % p_value)

Test statistic:				2.041 
95% percentile chi squared with 3 df:	7.815
p-value for different distributions:	0.564


Exercise 11

In [ ]:
df = pd.DataFrame({
    'Treatment': ['Placebo', 'Chlorpromazine', 'Dimenhydrate', 'Pentobarbital (100 mg)', 'Pentobarbital (150 mg)'],
    'Number of patients': [80, 75, 85, 67, 85],
    'Incidents of Nausea': [45, 26, 52, 35, 37]
})
df['fracs'] = df['Incidents of Nausea'] / df['Number of patients']
df['variance'] = df['fracs'] * (1 - df['fracs']) / df['Number of patients']

df['odds'] = df['fracs'] / (1 - df['fracs'])
df['odds ratios'] = df['odds'][1:] / df['odds'][0]
df['deltas'] = df['fracs'][1:] - df['fracs'][0]
df['stde'] = np.sqrt(df['variance'][1:] + df['variance'][0])
df['W'] = df['deltas'][1:] / df['stde'][1:]
df['p-value'] = 2 * norm.cdf(-np.abs(df['W']))
df['Bonferroni p-value'] = df['p-value']*4
df['BH threshold'] = df.index * 0.05 / 4
df[['Treatment', 'odds ratios', 'W', 'p-value','Bonferroni p-value', 'BH threshold']][df['Treatment'] != 'Placebo']

,Treatment,odds ratios,W,p-value,Bonferroni p-value,BH threshold
1,Chlorpromazine,0.412698,-2.764364,0.005703,0.022814,0.0125
2,Dimenhydrate,1.225589,0.642987,0.520232,2.080929,0.0250
3,Pentobarbital (100 mg),0.850694,-0.486428,0.626664,2.506656,0.0375
4,Pentobarbital (150 mg),0.599537,-1.646605,0.099639,0.398557,0.0500


Exercise 12

In [62]:
lambda_0 = 1
n = 20
alpha = 0.05
n_sims = 10000
z = norm.ppf(1 - alpha/2)
X = poisson.rvs(lambda_0, size = [n_sims, n])
lambda_hat = np.mean(X, axis = 1)
se = np.sqrt(lambda_0/n)
W = (lambda_hat - lambda_0)/se
n_reject = np.sum(np.abs(W) > z)
type_1 = n_reject / n_sims
type_1, len(W)

(0.0528, 10000)